# Mortgage Loan Risk Analysis at Scale with **PySpark**

## Business Problem (Case Study)
A large mortgage company processes **millions of loan applications** and monthly repayments. Risk analysis (defaults, delinquency, and early warning signals) is currently done in spreadsheets and single-machine tools, which do not scale. Decision-makers need **near real‑time** risk signals and **batch analytics** over years of history.

**Goal:** Use **PySpark** to build a scalable pipeline that:
- Ingests borrower, loan, and repayment data (historical + near real‑time)
- Cleans, unifies, and aggregates repayment behavior
- Engineers risk features (LTV, DTI, missed payments)
- Trains & evaluates a predictive model (MLlib) to flag **High/Medium/Low** risk loans
- Produces actionable outputs for underwriting and collections (CSV/Parquet/DB)

### Who uses this
- **Data Engineers**: Build distributed ETL & streaming pipelines
- **Data Scientists/ML Engineers**: Feature engineering & modeling at scale (MLlib)
- **Risk & Collections Teams**: Consume high‑risk flags & dashboards

### Why PySpark (vs Pandas)
- **Scale** to 100s of GBs / billions of rows with cluster execution
- **Batch + Streaming** in one engine (Spark SQL, Structured Streaming)
- **MLlib** for distributed model training & inference


## Environment Setup
Run the cell below if you're on Google Colab or a fresh environment.

> **Note:** If you're running locally and already have Spark, you can skip the install.

In [1]:
# If running in Colab or a fresh environment, uncomment to install PySpark
!pip install -q pyspark==3.5.1


## Create Spark Session
This initializes Spark in **local** mode for demos; switch to a cluster master (e.g., `yarn`, `k8s`) in production.

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("MortgageRiskAnalysis")
         .master("local[*]")
         .config("spark.sql.shuffle.partitions", "8")
         .getOrCreate())
spark


## Synthetic Data Generation (for demo)
We create three core datasets:

1. **Borrowers** – demographics & income
2. **Loans** – principal, property value, interest rate
3. **Repayments** – monthly payments with on‑time/delinquent flags

This is synthetic but **schema‑realistic** so you can plug in your real data later.

In [5]:
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window
import random

spark.conf.set("spark.sql.session.timeZone", "UTC")

N_BORROWERS = 100_000  # adjust downward if limited resources
N_LOANS = 120_000
N_REPAY = 1_000_000

# --- Borrowers ---
borrowers_schema = T.StructType([
    T.StructField("borrower_id", T.IntegerType(), False),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("annual_income", T.DoubleType(), True),
    T.StructField("credit_score", T.IntegerType(), True),
    T.StructField("state", T.StringType(), True),
])

states = ["CA", "TX", "NY", "FL", "WA", "IL", "PA", "OH", "GA", "NC"]
borrowers_rdd = spark.sparkContext.parallelize([
    (
        i,
        int(random.gauss(42, 10)) % 75 + 21,  # age 21..95
        float(max(20_000, random.gauss(95_000, 30_000))), # Cast to float/double
        int(max(300, min(850, random.gauss(690, 60)))),
        random.choice(states),
    )
    for i in range(1, N_BORROWERS + 1)
])
borrowers_df = spark.createDataFrame(borrowers_rdd, schema=borrowers_schema)
borrowers_df.cache()
print(borrowers_df.count())
borrowers_df.show(5, truncate=False)

# --- Loans ---
loans_schema = T.StructType([
    T.StructField("loan_id", T.IntegerType(), False),
    T.StructField("borrower_id", T.IntegerType(), False),
    T.StructField("principal", T.DoubleType(), True),
    T.StructField("property_value", T.DoubleType(), True),
    T.StructField("interest_rate", T.DoubleType(), True),
    T.StructField("origination_date", T.DateType(), True),
])

from datetime import date, timedelta
start_date = date(2018, 1, 1)

loans_rdd = spark.sparkContext.parallelize([
    (
        i,
        random.randint(1, N_BORROWERS),
        float(max(50_000, random.gauss(280_000, 90_000))), # Cast to float/double
        float(max(80_000, random.gauss(350_000, 120_000))), # Cast to float/double
        float(max(2.0, min(12.0, random.gauss(5.5, 1.5)))),
        start_date + timedelta(days=random.randint(0, 365*6)),
    )
    for i in range(1, N_LOANS + 1)
])
loans_df = spark.createDataFrame(loans_rdd, schema=loans_schema)
loans_df.cache()
print(loans_df.count())
loans_df.show(5, truncate=False)

# --- Repayments ---
repay_schema = T.StructType([
    T.StructField("repay_id", T.LongType(), False),
    T.StructField("loan_id", T.IntegerType(), False),
    T.StructField("due_date", T.DateType(), True),
    T.StructField("paid_date", T.DateType(), True),
    T.StructField("amount_due", T.DoubleType(), True),
    T.StructField("amount_paid", T.DoubleType(), True),
    T.StructField("status", T.StringType(), True),  # ON_TIME / LATE / MISSED
])

def random_repay(loan_id, start_date, n=12):
    rows = []
    for i in range(n):
        due = start_date + timedelta(days=30*(i+1))
        noise = random.randint(-5, 20)  # days late
        status_roll = random.random()
        if status_roll < 0.78:
            status = "ON_TIME"; paid = due + timedelta(days=max(0, noise))
            amt_paid = 1500.0 + random.uniform(-100, 100) # Ensure double type
        elif status_roll < 0.93:
            status = "LATE"; paid = due + timedelta(days=random.randint(6, 40))
            amt_paid = 1500.0 + random.uniform(-150, 0) # Ensure double type
        else:
            status = "MISSED"; paid = None
            amt_paid = 0.0 # Ensure double type
        # Use a placeholder value for repay_id initially
        rows.append((0, loan_id, due, paid, 1500.0, amt_paid, status)) # Ensure 1500 is double
    return rows

sample_loans = loans_df.select("loan_id", "origination_date").limit(int(N_REPAY/12)).collect()
all_repays = []
for row in sample_loans:
    all_repays.extend(random_repay(row.loan_id, row.origination_date, n=12))

repay_df = spark.createDataFrame(all_repays, schema=repay_schema)
repay_df = repay_df.withColumn("repay_id", F.monotonically_increasing_id())
repay_df.cache()
print(repay_df.count())
repay_df.show(5, truncate=False)

100000
+-----------+---+------------------+------------+-----+
|borrower_id|age|annual_income     |credit_score|state|
+-----------+---+------------------+------------+-----+
|1          |65 |128337.15796399579|813         |GA   |
|2          |51 |118043.90912342956|741         |FL   |
|3          |53 |105503.53052249731|674         |NC   |
|4          |62 |36018.43826081496 |728         |NY   |
|5          |45 |26668.4801683572  |677         |GA   |
+-----------+---+------------------+------------+-----+
only showing top 5 rows

120000
+-------+-----------+------------------+------------------+-----------------+----------------+
|loan_id|borrower_id|principal         |property_value    |interest_rate    |origination_date|
+-------+-----------+------------------+------------------+-----------------+----------------+
|1      |93099      |202623.46368091126|279313.44673850277|4.939162172074387|2018-01-23      |
|2      |81799      |191572.73627576535|559554.0934648707 |3.867899816064522|

## ETL: Clean, Join, and Aggregate
We unify **borrowers + loans + repayments** and calculate payment behavior metrics.

In [6]:
loans_enriched = (loans_df
    .withColumn("ltv", F.col("principal")/F.col("property_value"))  # loan-to-value
)

repay_agg = (repay_df
    .groupBy("loan_id")
    .agg(
        F.sum(F.when(F.col("status")=="MISSED", 1).otherwise(0)).alias("missed_payments"),
        F.sum(F.when(F.col("status")=="LATE", 1).otherwise(0)).alias("late_payments"),
        F.sum("amount_paid").alias("total_paid"),
        F.count("repay_id").alias("n_payments")
    )
)

unified = (loans_enriched
    .join(repay_agg, on="loan_id", how="left")
    .join(borrowers_df, on="borrower_id", how="left")
    .fillna({"missed_payments":0, "late_payments":0, "total_paid":0.0, "n_payments":0})
    .withColumn("dti", (F.col("principal")*F.col("interest_rate")/100.0) / F.col("annual_income"))
    .withColumn("is_delinquent", (F.col("missed_payments") > 0).cast("int"))
)

unified.select("loan_id","borrower_id","ltv","dti","missed_payments","late_payments","credit_score","annual_income").show(10, truncate=False)


+-------+-----------+-------------------+-------------------+---------------+-------------+------------+------------------+
|loan_id|borrower_id|ltv                |dti                |missed_payments|late_payments|credit_score|annual_income     |
+-------+-----------+-------------------+-------------------+---------------+-------------+------------+------------------+
|6      |44075      |1.7611839792862642 |0.1708622283416246 |1              |1            |630         |100284.64544591069|
|9      |19261      |1.2117409944280189 |0.19541905047965122|3              |2            |715         |105123.2502761841 |
|5      |40142      |1.5270576953535762 |0.2731405543470082 |1              |0            |696         |63003.744583037245|
|10     |2131       |0.6290050861210621 |0.18752689983719092|1              |0            |664         |84201.08446631787 |
|1      |93099      |0.725433974077912  |0.09660617565036222|0              |2            |563         |103594.84165996926|
|3      

## Feature Engineering
We build risk features used by ML models:
- `ltv` (loan-to-value)
- `dti` (debt-to-income proxy)
- `missed_payments`, `late_payments`
- `credit_score`
- Categorical encoding for `state`


In [7]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

label_col = "is_delinquent"  # binary classification target
cat_cols = ["state"]
num_cols = ["ltv", "dti", "missed_payments", "late_payments", "credit_score", "annual_income", "interest_rate"]

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]
encoders = [OneHotEncoder(inputCols=[f"{c}_idx"], outputCols=[f"{c}_oh"]) for c in cat_cols]

assembler = VectorAssembler(
    inputCols=[*num_cols, *[f"{c}_oh" for c in cat_cols]],
    outputCol="features"
)

pipeline = Pipeline(stages=[*indexers, *encoders, assembler])
feat_model = pipeline.fit(unified)
data_ml = feat_model.transform(unified).select("features", label_col)
data_ml.show(5, truncate=False)


+----------------------------------------------------------------------------------------------------------------------+-------------+
|features                                                                                                              |is_delinquent|
+----------------------------------------------------------------------------------------------------------------------+-------------+
|(17,[0,1,3,4,5,6,15],[0.8761142321318093,0.14456955571046973,2.0,759.0,113738.12559015868,5.108514588311969,1.0])     |0            |
|(17,[0,1,2,3,4,5,6,16],[0.8859900533340481,0.1500883483464581,1.0,2.0,635.0,123878.60294494792,5.832411084057021,1.0])|1            |
|(17,[0,1,4,5,6,7],[1.0737379381228682,0.09210849356968505,603.0,138230.0009834121,5.0163487897103005,1.0])            |0            |
|(17,[0,1,2,3,4,5,6,10],[0.9520619397265584,0.2394427473455542,1.0,1.0,721.0,84714.46906925566,7.100918920975623,1.0]) |1            |
|(17,[0,1,2,4,5,6,15],[1.5610886512615325,0.08221323444

## Train/Test Split & Models (MLlib)
We train two classifiers for comparison:
- **Logistic Regression**
- **Random Forest**

In [8]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

train, test = data_ml.randomSplit([0.75, 0.25], seed=42)

lr = LogisticRegression(labelCol=label_col, featuresCol="features", maxIter=20)
rf = RandomForestClassifier(labelCol=label_col, featuresCol="features", numTrees=60, maxDepth=8)

lr_model = lr.fit(train)
rf_model = rf.fit(train)

lr_pred = lr_model.transform(test)
rf_pred = rf_model.transform(test)

evaluator = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")
lr_auc = evaluator.evaluate(lr_pred)
rf_auc = evaluator.evaluate(rf_pred)

print(f"Logistic Regression AUC: {lr_auc:.4f}")
print(f"Random Forest AUC: {rf_auc:.4f}")


Logistic Regression AUC: 1.0000
Random Forest AUC: 1.0000


## (Optional) Structured Streaming Demo
Below is a **template** that shows how you would process incremental repayment events via Structured Streaming. For a quick local demo, you can use the `rate` source or watch a directory for new CSV files.

> **Note:** This section is illustrative; adapt to your storage (Kafka, Kinesis, files) and checkpoint locations.

In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, DoubleType, StringType

# Example schema for streaming repayments
stream_schema = StructType([
    StructField("loan_id", IntegerType()),
    StructField("due_date", DateType()),
    StructField("paid_date", DateType()),
    StructField("amount_due", DoubleType()),
    StructField("amount_paid", DoubleType()),
    StructField("status", StringType()),
])

# Example: read from a folder where new CSV files land
# stream_df = (spark.readStream
#     .schema(stream_schema)
#     .option("header", True)
#     .csv("/path/to/incoming/repayments")
# )

# agg_stream = (stream_df
#     .groupBy("loan_id")
#     .agg(F.sum(F.when(F.col("status")=="MISSED", 1).otherwise(0)).alias("missed_payments"))
# )

# query = (agg_stream.writeStream
#     .format("memory")  # for demo; use parquet/delta/kafka in prod
#     .queryName("repay_agg_live")
#     .outputMode("complete")
#     .option("checkpointLocation", "/tmp/checkpoints/repay_agg_live")
#     .start()
# )

# # Later, query the live table
# spark.sql("select * from repay_agg_live").show()


## Interpretability & Next Steps
- Add **calibration** & business‑friendly scorecards (e.g., PD bands)
- Track **drift** in repayment behavior & credit score distribution
- Move training data & models to **Delta Lake** with versioning
- Serve predictions via **batch scoring** or **real‑time microservices** (e.g., Spark on Kubernetes + REST)
- Add **fairness** checks & bias monitoring across states/segments

### Key Differences vs Pandas
- PySpark executes lazily and **distributes** computation across a cluster
- Can handle **100M+ rows** with both batch and streaming in one engine
- Integrates with **MLlib**, **SQL**, **Delta/Parquet**, **HDFS/S3/ADLS**, and BI tools
